In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import pickle

In [2]:
accepted_df = pd.read_csv("../Resources/lending_club_accepted.csv")
accepted_df.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,disbursement_method
0,32000.0,60 months,10.49,687.65,10+ years,MORTGAGE,120000.0,Verified,Current,n,...,4.0,100.0,28.6,0.0,0.0,556496.0,103647.0,64100.0,72197.0,Cash
1,9600.0,36 months,12.99,323.42,NaN,RENT,21900.0,Verified,Fully Paid,n,...,2.0,100.0,50.0,1.0,0.0,11600.0,4509.0,2400.0,0.0,Cash
2,4000.0,36 months,6.68,122.93,4 years,MORTGAGE,83000.0,Not Verified,Fully Paid,n,...,2.0,100.0,0.0,0.0,0.0,222616.0,64253.0,5600.0,76154.0,Cash
3,6025.0,36 months,10.91,197.00,10+ years,RENT,52000.0,Not Verified,Fully Paid,n,...,0.0,96.0,0.0,0.0,0.0,32227.0,5559.0,11000.0,11127.0,Cash
4,25000.0,60 months,26.30,752.96,10+ years,OWN,65000.0,Verified,Current,n,...,3.0,78.8,10.0,0.0,0.0,257219.0,97647.0,179400.0,65719.0,Cash


In [3]:
rejected_df = pd.read_csv("../Resources/lending_club_rejected.csv")
rejected_df.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,15000.0,2015-10-29,debt_consolidation,NaN,54.58%,761xx,TX,< 1 year,0.0
1,10000.0,2017-10-28,Credit card refinancing,605.0,28.71%,076xx,NJ,NaN,0.0
2,10000.0,2018-03-09,Debt consolidation,NaN,58.06%,087xx,NJ,< 1 year,0.0
3,7000.0,2016-10-18,Debt consolidation,571.0,6.66%,731xx,OK,< 1 year,0.0
4,5000.0,2017-02-07,other,NaN,2.23%,537xx,WI,< 1 year,0.0


In [4]:
accepted_df_2 = accepted_df[['loan_amnt', 'title', 'dti', 'zip_code','addr_state', 'emp_length', 'policy_code']].rename(columns = {
    'loan_amnt': 'Amount Requested',
    'title': 'Loan Title',
    'dti': 'Debt-To-Income Ratio',
    'zip_code': 'Zip Code',
    'addr_state': 'State',
    'emp_length': 'Employment Length',
    'policy_code': 'Policy Code'
})
accepted_df_2.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,32000.0,Debt consolidation,24.05,919xx,CA,10+ years,1.0
1,9600.0,Debt consolidation,10.03,331xx,FL,NaN,1.0
2,4000.0,Major purchase,19.53,333xx,FL,4 years,1.0
3,6025.0,Debt consolidation,9.16,021xx,MA,10+ years,1.0
4,25000.0,Debt consolidation,36.26,926xx,CA,10+ years,1.0


In [5]:
rejected_df_2 = rejected_df.drop(['Application Date', 'Risk_Score'], axis=1)
rejected_df_2.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,15000.0,debt_consolidation,54.58%,761xx,TX,< 1 year,0.0
1,10000.0,Credit card refinancing,28.71%,076xx,NJ,NaN,0.0
2,10000.0,Debt consolidation,58.06%,087xx,NJ,< 1 year,0.0
3,7000.0,Debt consolidation,6.66%,731xx,OK,< 1 year,0.0
4,5000.0,other,2.23%,537xx,WI,< 1 year,0.0


In [6]:
len(accepted_df_2)

56518

In [7]:
len(rejected_df_2)

691219

In [8]:
rejected_sample = rejected_df_2.sample(frac=0.0817656922, random_state=42)
len(rejected_sample)

56518

In [9]:
# Some policy codes are 2.
# https://news.fintechnexus.com/policy-code-2-loans-lending-club/
# policy code 0 means rejected
# policy code 1 means accepted
# policy code 2 means accepted, but considered sub-prime on credit rating

# We're merely predicting whether or not someone's loan will be accepted or rejected, 
# so we'll change the status and simplify our model
rejected_sample['Policy Code'] = 0
accepted_df_2['Policy Code'] = 1

In [10]:
loan_application_df = pd.DataFrame.append(accepted_df_2, rejected_sample)

len(loan_application_df)

113036

In [11]:
loan_application_df.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,32000.0,Debt consolidation,24.05,919xx,CA,10+ years,1
1,9600.0,Debt consolidation,10.03,331xx,FL,NaN,1
2,4000.0,Major purchase,19.53,333xx,FL,4 years,1
3,6025.0,Debt consolidation,9.16,021xx,MA,10+ years,1
4,25000.0,Debt consolidation,36.26,926xx,CA,10+ years,1


In [12]:
len(loan_application_df.dropna())

107013

In [13]:
loan_application_df = loan_application_df.dropna()

In [14]:
loan_application_df.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,32000.0,Debt consolidation,24.05,919xx,CA,10+ years,1
2,4000.0,Major purchase,19.53,333xx,FL,4 years,1
3,6025.0,Debt consolidation,9.16,021xx,MA,10+ years,1
4,25000.0,Debt consolidation,36.26,926xx,CA,10+ years,1
5,20000.0,Debt consolidation,16.43,891xx,NV,10+ years,1


In [15]:
loan_state = loan_application_df.drop(['Zip Code'], axis=1)
loan_state.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,State,Employment Length,Policy Code
0,32000.0,Debt consolidation,24.05,CA,10+ years,1
2,4000.0,Major purchase,19.53,FL,4 years,1
3,6025.0,Debt consolidation,9.16,MA,10+ years,1
4,25000.0,Debt consolidation,36.26,CA,10+ years,1
5,20000.0,Debt consolidation,16.43,NV,10+ years,1


In [16]:
loan_state = loan_state.astype({
    'Policy Code': int,
    'Amount Requested': int
})
loan_state.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,State,Employment Length,Policy Code
0,32000,Debt consolidation,24.05,CA,10+ years,1
2,4000,Major purchase,19.53,FL,4 years,1
3,6025,Debt consolidation,9.16,MA,10+ years,1
4,25000,Debt consolidation,36.26,CA,10+ years,1
5,20000,Debt consolidation,16.43,NV,10+ years,1


In [17]:
X = loan_state.drop(['Policy Code'], axis=1)
X.head()

,Amount Requested,Loan Title,Debt-To-Income Ratio,State,Employment Length
0,32000,Debt consolidation,24.05,CA,10+ years
2,4000,Major purchase,19.53,FL,4 years
3,6025,Debt consolidation,9.16,MA,10+ years
4,25000,Debt consolidation,36.26,CA,10+ years
5,20000,Debt consolidation,16.43,NV,10+ years


In [18]:
y = loan_state['Policy Code']
y.head()

0    1
2    1
3    1
4    1
5    1
Name: Policy Code, dtype: int32

In [19]:
X_dummies = pd.get_dummies(X)
# Scale the data
scaler = StandardScaler().fit(X_dummies)
X_scaled = scaler.transform(X_dummies)
# X_train_scaled


In [20]:
filename = 'loan_acceptance_scaler.sav'
pickle.dump(scaler, open(filename, 'wb'))

In [21]:
X_scaled

array([[ 1.39292486, -0.01729504, -0.00305692, ..., -0.14990849,
        -0.14054944, -0.96393475],
       [-0.80117121, -0.01729504, -0.00305692, ..., -0.14990849,
        -0.14054944, -0.96393475],
       [-0.64249105, -0.01729504, -0.00305692, ..., -0.14990849,
        -0.14054944, -0.96393475],
       ...,
       [-0.72281063, -0.01729504, -0.00305692, ..., -0.14990849,
        -0.14054944, -0.96393475],
       [-0.17428662, -0.01729504, -0.00305692, ..., -0.14990849,
        -0.14054944,  1.03741462],
       [ 0.0607951 , -0.01729504, -0.00305692, ..., -0.14990849,
        -0.14054944,  1.03741462]])

In [22]:
def confusion_score(y_true, y_pred):
    [[TP, FN],[FP,TN]] = confusion_matrix(y_true, y_pred)

    accuracy = (TP + TN) / (TP + FP + TN + FN) # (111 + 128) / (111 + 5 + 128 + 6)
    print(f"Accuracy: {accuracy.round(4)}")
    precision = TP / (TP + FP)
    print(f'Precision: {precision.round(4)}')
    sensitivity = TP / (TP + FN)
    print(f'Sensitivity: {sensitivity.round(4)}')
    specificity = TN / (TN + FP)
    print(f'Specificity: {specificity.round(4)}')

In [23]:
def confusion_score2(y_true, y_pred):
    [[a1, a2, a3],[b1,b2, b3],[c1,c2,c3]] = confusion_matrix(y_true, y_pred)
    TP = a1
    FN = a2 + a3
    FP = b1 + c1
    TN = b2 + b3 + c2 + c3

    accuracy = (TP + TN) / (TP + FP + TN + FN) # (111 + 128) / (111 + 5 + 128 + 6)
    print(f"Accuracy: {accuracy.round(3)}")
    precision = TP / (TP + FP)
    print(f'Precision: {precision.round(3)}')
    sensitivity = TP / (TP + FN)
    print(f'Sensitivity: {sensitivity.round(3)}')
    specificity = TN / (TN + FP)
    print(f'Specificity: {specificity.round(3)}')

In [24]:
# split the data into test and train

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=1)

In [25]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9985422740524781


In [26]:
y_true = y_test
y_pred = classifier.predict(X_test)
print('Confusion Matrix:\n',confusion_matrix(y_true, y_pred), '\n')

confusion_score(y_true, y_pred)

Confusion Matrix:
 [[13570    11]
 [   28 13145]] 

Accuracy: 0.9985
Precision: 0.9979
Sensitivity: 0.9992
Specificity: 0.9979


In [27]:
y.unique()

array([1, 0])

In [28]:
y.value_counts()

0    54566
1    52447
Name: Policy Code, dtype: int64

#### We suspect that employment length is the largest differentiator between a loan application being accepted or rejected.

In [29]:
rejected_sample['Employment Length'].value_counts()

< 1 year     46876
5 years       4678
10+ years      903
1 year         551
2 years        423
3 years        385
4 years        261
6 years        144
8 years        127
7 years        118
9 years        101
Name: Employment Length, dtype: int64

In [30]:
accepted_df_2['Employment Length'].value_counts()

10+ years    18695
2 years       5081
< 1 year      4683
3 years       4505
1 year        3753
5 years       3606
4 years       3411
6 years       2562
7 years       2389
8 years       2258
9 years       1981
Name: Employment Length, dtype: int64

#### This seems to be a reasonable assumption.

In [32]:
filename = 'loan_acceptance_model.sav'
pickle.dump(classifier, open(filename, 'wb'))